In [4]:
from uncertainty_engine import Client

client = Client(env="dev")
client.authenticate()

In [ ]:
project_id = client.projects.get_project_id_by_name("Personal2")
print(project_id)

resource_id = client.resources.get_resource_id_by_name(project_id, "document", "e2e_test_dataset")
print(resource_id)

In [ ]:
from pprint import pprint

# Print the details of the `TrainModel` node
pprint(client.get_node_info("TrainModel").model_dump())

In [ ]:
import numpy as np
import pandas as pd

# Set random seed for reproducibility and assign number of rows
np.random.seed(42)
n_rows = 10

# Target function: forrester function
def f(x, a=6, b=12):
    return -((a * x - 2) ** 2) * np.sin(b * x - 4)

# Generate synthetic data to train on
X_train = np.random.uniform(0, 1, size=n_rows) # 10 randomly sampled points
y_train = f(X_train) # Place those points into the target function
train_df = pd.DataFrame({
    'x': X_train,
    'y': y_train
})

# Generate "truth" data for testing
X_test = np.linspace(0, 1, 100)
y_test = f(X_test)
test_df = pd.DataFrame({"x": X_test, "y": y_test})

In [ ]:
# Convert dataframes to CSV strings
x_csv = train_df[["x"]].to_csv(index=False)
y_csv = train_df[["y"]].to_csv(index=False)
x_test_csv = test_df[["x"]].to_csv(index=False)

# Save CSV strings to files
for filename, data in zip(
    ["x_train.csv", "y_train.csv", "x_test.csv"], [x_csv, y_csv, x_test_csv]
):
    with open(filename, "w") as f:
        f.write(data)

In [ ]:
# List all available projects
projects = client.projects.list_projects()

# Find the project named "Personal"
personal_project = next(p for p in projects if p.name == "Personal")

In [ ]:
# Define an empty dictionary to hold resource IDs
resource_ids: dict[str,str] = dict()

# Upload datasets and store their resource IDs
for resource_name, file_path in zip(
    ["my-x-dataset", "my-y-dataset", "my-x-test-dataset"],
    ["x_train.csv", "y_train.csv", "x_test.csv"],
):
    resource_id = client.resources.upload(
        project_id=personal_project.id,
        name=resource_name,
        file_path=file_path,
        resource_type="dataset",
    )
    resource_ids[resource_name] = resource_id
    print(f"Uploaded {file_path} as resource '{resource_name}'")

In [ ]:
pprint(client.get_node_info("ModelConfig").model_dump())

In [ ]:
from uncertainty_engine.graph import Graph

# Create a new graph
train_graph = Graph()

In [ ]:
from uncertainty_engine.nodes.machine_learning import ModelConfig

# Define the model config node
model_config = ModelConfig(
    label="Model Config",
)

# Add node to the graph and connect it to the train node
train_graph.add_node(model_config)

In [ ]:
from uncertainty_engine.nodes.resource_management import LoadDataset

# Create a LoadDataset node for the x training data
x = LoadDataset(
  label="LoadDatasetX", # the node's label (needs to be unique in the graph)
  project_id = personal_project.id, # the project id
  file_id = resource_ids["my-x-dataset"], # the resource id of the file you want to load
)
# Create a LoadDataset node for the y training data
y = LoadDataset(
  label="LoadDatasetY", # the node's label (needs to be unique in the graph)
  project_id = personal_project.id, # the project id
  file_id = resource_ids["my-y-dataset"], # the resource id of the file you want to load
)

train_graph.add_node(x)
train_graph.add_node(y)

In [ ]:
from uncertainty_engine.nodes.machine_learning import TrainModel

# Create handles for the configuration and loaded dataset files
output_config = model_config.make_handle("config")
x_handle = x.make_handle("file")
y_handle = y.make_handle("file")

# Create the TrainModel node
train_model = TrainModel(
    label="Train Model",
    config=output_config,
    inputs=x_handle,
    outputs=y_handle,
)

# Add the node to the graph
train_graph.add_node(train_model)

In [ ]:
from uncertainty_engine.nodes.resource_management import Save

save_model = Save(
    label="SaveModel",
    data=train_model.make_handle("model"),
    project_id=personal_project.id,
    file_name="my-machine-learning-model",
)

train_graph.add_node(save_model)

In [ ]:
from uncertainty_engine.nodes.workflow import Workflow

# Wrap the graph in a workflow node
train_workflow = Workflow(
    graph=train_graph.nodes,
    inputs=train_graph.external_input,
    external_input_id=train_graph.external_input_id,
    )

In [ ]:
train_response = client.run_node(train_workflow)
assert train_response.status.value == "completed"  # Ensure the workflow completed successfully

In [ ]:
for model in client.resources.list_resources(project_id=personal_project.id, resource_type="model"):
    if model.name == "my-machine-learning-model":
        print(f"Model saved with resource ID: {model.id}")
        break  # Stop after finding the first match. We can use the model ID for future reference

In [ ]:
pprint(client.get_node_info("PredictModel").model_dump())

In [ ]:
predict_graph = Graph()

In [ ]:
x_test_node = LoadDataset(
    label="LoadDatasetXTest",
    project_id=personal_project.id,
    file_id=resource_ids["my-x-test-dataset"],
)
predict_graph.add_node(x_test_node)

In [ ]:
from uncertainty_engine.nodes.resource_management import LoadModel

load_model = LoadModel(
    label="LoadModel",
    project_id=personal_project.id,
    file_id=model.id,
)

predict_graph.add_node(load_model)

In [ ]:
from uncertainty_engine.nodes.machine_learning import PredictModel

predict = PredictModel(
    label="Predict",
    dataset=x_test_node.make_handle("file"),
    model=load_model.make_handle("file"),
)
predict_graph.add_node(predict)

In [ ]:
from uncertainty_engine.nodes.resource_management import Download

# Add handles to the prediction and uncertainty outputs
output_predictions = predict.make_handle("prediction")
output_uncertainty = predict.make_handle("uncertainty")

# Define download nodes for predictions and uncertainty
download_predictions = Download(
    label="Download Predictions",
    file=output_predictions
)
download_uncertainty = Download(
    label="Download Uncertainty",
    file=output_uncertainty
)

# Add download nodes to the graph and connect them to the predict node
predict_graph.add_node(download_predictions)
predict_graph.add_node(download_uncertainty)

In [ ]:
# Define the output handles for the download nodes
output_download_predictions = download_predictions.make_handle("file")
output_download_uncertainty = download_uncertainty.make_handle("file")

# Wrap the predict graph in a workflow node
predict_workflow = Workflow(
    graph=predict_graph.nodes,
    inputs=predict_graph.external_input,
    external_input_id=predict_graph.external_input_id,
    requested_output={
        "Predictions": output_download_predictions.model_dump(),
        "Uncertainty": output_download_uncertainty.model_dump(),
        }
    )

# Run the predict workflow and get the response
predict_response = client.run_node(predict_workflow)

In [ ]:
from io import StringIO
import requests

def get_presigned_url(url):
    """
    Get the contents from the presigned url.
    """
    url = url.replace("https://", "http://")
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors
    return response

# Download the predictions and save as a DataFrame
predictions_response = get_presigned_url(predict_response.outputs["outputs"]["Predictions"])
predictions_df = pd.read_csv(StringIO(predictions_response.text))  # Save the predictions to a DataFrame

# Download the uncertainty and save as a DataFrame
uncertainty_response = get_presigned_url(predict_response.outputs["outputs"]["Uncertainty"])
uncertainty_df = pd.read_csv(StringIO(uncertainty_response.text))  # Save the uncertainty to a DataFrame

In [ ]:
for resource_id in resource_ids.values():
    client.resources.delete_resource(
        project_id=personal_project.id,
        resource_id=resource_id,
        resource_type="dataset",
    )

In [ ]:
client.resources.delete_resource(
    project_id=personal_project.id,
    resource_id=model.id,
    resource_type="model",
)